# Tool Use

Tool use, also known as function calling, is a key feature of agentic applications. 

In [9]:
from langchain_dartmouth.llms import ChatDartmouthCloud
from dotenv import find_dotenv, load_dotenv

load_dotenv(find_dotenv())

True

[Dog API](https://dogapi.dog/docs/api-v2)

In [10]:
from langchain_core.tools import tool

import requests

import os


@tool
def search_breed(breed: str) -> list[dict[str, str | int]]:
    """Search the database for information on a given dog breed. Weight is in pounds, height in inches. All scales range from 1 (lowest) to 5 (highest)."""
    response = requests.get(
        url="https://api.api-ninjas.com/v1/dogs",
        params={"name": breed},
        headers={"x-api-key": os.environ["DOG_API_KEY"]},
    )
    return response.json()


tools = [search_breed]

In [11]:
llm = ChatDartmouthCloud(model_name="openai.gpt-4.1-mini-2025-04-14")
llm_with_tools = llm.bind_tools(tools)

In [12]:
from langchain_core.messages import HumanMessage

messages = [HumanMessage("What can you tell me about the dachshund?")]
response = llm_with_tools.invoke(messages)
messages.append(response)

if tool_calls := response.tool_calls:
    for tool_call in tool_calls:
        fn, *_ = [tool for tool in tools if tool.name == tool_call["name"]]
        tool_msg = fn.invoke(tool_call)
        messages.append(tool_msg)
response = llm_with_tools.invoke(messages)
messages.append(response)

for message in messages:
    message.pretty_print()

================================ Human Message =================================

What can you tell me about the dachshund?
================================== Ai Message ==================================
Tool Calls:
  search_breed (call_MiFP4NxzdFqIxXGjseTObHWY)
 Call ID: call_MiFP4NxzdFqIxXGjseTObHWY
  Args:
    breed: dachshund
================================= Tool Message =================================
Name: search_breed

[{"image_link": "https://api-ninjas.com/images/dogs/dachshund.jpg", "good_with_children": 3, "good_with_other_dogs": 4, "shedding": 2, "grooming": 2, "drooling": 2, "coat_length": 2, "good_with_strangers": 4, "playfulness": 4, "protectiveness": 4, "trainability": 4, "energy": 3, "barking": 5, "min_life_expectancy": 12.0, "max_life_expectancy": 16.0, "max_height_male": 9.0, "max_height_female": 9.0, "max_weight_male": 32.0, "max_weight_female": 32.0, "min_height_male": 8.0, "min_height_female": 8.0, "min_weight_male": 16.0, "min_weight_female": 16.0, "name": "D

## Summary

Tool use or function calling describes the ability of a Large Language Model to output arguments for functions whose output may help to further the user's goals.